**Data Creation**

In [0]:
# CONFIGURING BLOB ENDPOINT AUTH
STORAGE_ACCOUNT = "stfrriskneu"
ACCOUNT_KEY = "ISts3E65JSUeel9g1NdZ1XFbXxfPMpJzKeaqLb+CwW9lManoqTs+jHRq9WjodebD1ARdkxj/eX/N+ASt/H7fpQ=="
# Registering the key for the Blob endpoint
spark.conf.set(f"fs.azure.account.key.{STORAGE_ACCOUNT}.blob.core.windows.net", ACCOUNT_KEY)
# Base path at the CONTAINER root (do NOT append /bronze twice)
BLOB_BASE = f"wasbs://bronze@{STORAGE_ACCOUNT}.blob.core.windows.net"
print("Writing to:", BLOB_BASE)


Writing to: wasbs://bronze@stfrriskneu.blob.core.windows.net


In [0]:
# VOLUME + "DIRTY DATA" KNOBS
N_CUSTOMERS   = 50_000
TXN_PER_MONTH = 300_000
FRAUD_RATE    = 0.025   # ~2.5% overall label

NULL_PCT_SOFT = 0.03    # ~3% missing values
TYPO_PCT      = 0.01    # ~1% random string typos

import random, uuid
random.seed(42)

from pyspark.sql import functions as F, types as T

def uid(prefix=""): 
    return prefix + uuid.uuid4().hex[:16]

def maybe_null(col, p=0.03):
    return F.when(F.rand() < F.lit(p), F.lit(None)).otherwise(col)

def introduce_typo(col, p=0.01):
    # simple swap of letters to mimic user typo
    return F.when((F.rand() < F.lit(p)) & (F.length(col) > 2),
                  F.concat_ws("", F.substring(col,1,2), F.substring(col,2,1), F.substring(col,3,1000))
                 ).otherwise(col)

def dirty(df, null_p=0.03, typo_p=0.01, protected=("id","is_fraud","is_high_amount","ip")):
    # Add nulls/typos to non-ID, non-label columns
    for name, dtype in df.dtypes:
        if any(k in name.lower() for k in protected):
            continue
        df = df.withColumn(name, maybe_null(F.col(name), null_p))
        if dtype == "string":
            df = df.withColumn(name, introduce_typo(F.col(name), typo_p))
    return df

# REFERENCE DOMAINS
countries = [("IE","Dublin"),("GB","London"),("FR","Paris"),("DE","Berlin"),
             ("ES","Madrid"),("NL","Amsterdam"),("IT","Milan"),("PL","Warsaw"),
             ("US","New York"),("IN","Mumbai")]
mccs        = ["5411","5812","5814","5311","5944","5732","4111","6011","4829","7995"]
channels    = ["POS","WEB","MOBILE"]
entry_modes = ["CHIP","SWIPE","TAP","KEYED"]
risk_buckets= ["LOW","MED","HIGH"]
os_list     = ["iOS","Android","Windows","Linux","macOS"]


In [0]:
# MERCHANTS
merchants = (spark.range(10_000)
 .withColumn("merchant_id", F.expr("uuid()"))
 .withColumn("category_mcc", F.element_at(F.array(*[F.lit(x) for x in mccs]), (F.rand()*len(mccs)).cast("int")+1))
 .withColumn("risk_bucket",  F.element_at(F.array(*[F.lit(x) for x in risk_buckets]), (F.rand()*len(risk_buckets)).cast("int")+1))
 .withColumn("country_iso",  F.element_at(F.array(*[F.lit(c[0]) for c in countries]), (F.rand()*len(countries)).cast("int")+1))
 .withColumn("avg_ticket",   F.round(F.rand()*200 + 10, 2))
 .withColumn("velocity_cap", (F.rand()*50+5).cast("int")))

#Later correction(after noticing problem in PBI)
from pyspark.sql import Window

w_merch = Window.orderBy("merchant_id")

merchants_idx = (
    merchants
    .withColumn("merchant_idx", F.row_number().over(w_merch) - 1)
)

N_MERCHANTS = merchants_idx.count()
print("Number of merchants:", N_MERCHANTS)

# DEVICES
devices = (spark.range(60_000)
 .withColumn("device_id", F.expr("uuid()"))
 .withColumn("device_type", F.when(F.rand()<0.7,"Mobile").when(F.rand()<0.9,"Web").otherwise("POS"))
 .withColumn("os", F.element_at(F.array(*[F.lit(o) for o in os_list]), (F.rand()*len(os_list)).cast("int")+1))
 .withColumn("first_seen_dt",
   F.from_unixtime(F.unix_timestamp() - (F.rand() * (365*24*60*60)).cast("int")).cast("timestamp"))
 .withColumn("fingerprint_hash", F.expr("sha1(uuid())"))
 .withColumn("is_rooted", (F.rand()<0.03).cast("boolean")))

# ACCOUNTS
accounts = (spark.range(N_CUSTOMERS)
 .withColumn("account_id", F.expr("uuid()"))
 .withColumn("customer_name", F.concat(F.lit("User-"), F.col("id")))
 .withColumn("email", F.concat(F.lit("user"), F.col("id"), F.lit("@example.com")))
 .withColumn("country_iso", F.element_at(F.array(*[F.lit(c[0]) for c in countries]), (F.rand()*len(countries)).cast("int")+1))
 .withColumn("created_dt",
   F.from_unixtime(F.unix_timestamp() - (F.rand() * (730*24*60*60)).cast("int")).cast("timestamp"))
 .withColumn("status", F.when(F.rand()<0.95,"ACTIVE").otherwise("BLOCKED")))

# IP REPUTATION
ip_rep = (spark.range(100_000)
 .withColumn("ip", F.concat((F.rand()*255).cast("int"), F.lit("."), (F.rand()*255).cast("int"),
                             F.lit("."), (F.rand()*255).cast("int"), F.lit("."), (F.rand()*255).cast("int")))
 .withColumn("asn", F.concat(F.lit("AS"), (F.rand()*50000).cast("int")))
 .withColumn("proxy_type", F.when(F.rand()<0.1,"VPN").when(F.rand()<0.15,"Proxy")
                            .when(F.rand()<0.17,"TOR").otherwise("Residential"))
 .withColumn("reputation_score", F.round(F.rand()*100,2))
 .withColumn("updated_dt",
   F.from_unixtime(F.unix_timestamp() - (F.rand() * (30*24*60*60)).cast("int")).cast("timestamp")))

#Addition after issue in PBI
from pyspark.sql import Window

w_ip = Window.orderBy("ip")

ip_rep_idx = (
    ip_rep
    .withColumn("ip_idx", F.row_number().over(w_ip) - 1)  # 0..N_IPS-1
)

N_IPS = ip_rep_idx.count()
print("Number of IPs:", N_IPS)



Number of merchants: 10000
Number of IPs: 100000


In [0]:
print('Hey')

Hey


In [0]:
# Sample to control volume
a = accounts.sample(False, 0.35).alias("a").cache()
m = merchants.sample(False, 0.30).alias("m").cache()

# TRANSACTIONS
transactions = (
    a.crossJoin(m.limit(TXN_PER_MONTH // max(1, int(N_CUSTOMERS*0.35))))
    .select(
        F.expr("uuid()").alias("txn_id"),
        F.col("a.account_id").alias("account_id"),
        F.concat(F.lit("CARD_"), F.monotonically_increasing_id().cast("string")).alias("card_id"),
        F.expr("uuid()").alias("device_id"),
        F.col("m.merchant_id").alias("merchant_id"),
        F.from_unixtime(F.unix_timestamp() - (F.rand() * (90*24*60*60)).cast("int")).cast("timestamp").alias("txn_dt"),
        F.round(F.rand()*500 + 5, 2).alias("amount"),
        F.lit("EUR").alias("currency"),
        F.col("m.country_iso").alias("country_iso"),
        F.element_at(F.array(*[F.lit(c[1]) for c in countries]), (F.rand()*len(countries)).cast("int")+1).alias("city"),
        F.element_at(F.array(*[F.lit(x) for x in channels]), (F.rand()*len(channels)).cast("int")+1).alias("channel"),
        F.element_at(F.array(*[F.lit(x) for x in entry_modes]), (F.rand()*len(entry_modes)).cast("int")+1).alias("entry_mode"),
        F.concat((F.rand()*255).cast("int"), F.lit("."),
                 (F.rand()*255).cast("int"), F.lit("."),
                 (F.rand()*255).cast("int"), F.lit("."),
                 (F.rand()*255).cast("int")).alias("ip"),
        F.round(F.rand()*1200,2).alias("distance_km_prev"),
        (F.rand()*6).cast("int").alias("txn_in_last_min5"),
        (F.rand()<0.08).cast("boolean").alias("is_international"),
        (F.rand()<0.12).cast("boolean").alias("is_new_device"),
        (F.rand()<0.22).cast("boolean").alias("is_night")
    )
    .withColumn("is_high_amount", (F.col("amount") > 250).cast("boolean"))
    .withColumn(
        "fraud_score",
        1.2*F.col("is_high_amount").cast("int")
      + 1.0*F.col("is_night").cast("int")
      + 1.1*F.col("is_international").cast("int")
      + 1.1*F.col("is_new_device").cast("int")
      + F.rand()*0.5
    )
    .withColumn("is_fraud", (F.col("fraud_score") > 2.6).cast("boolean"))
    .drop("fraud_score")
)

#Later correction(after noticing problem in PBI)
transactions = (
    transactions
        .drop("merchant_id")  # remove the existing one (even though it already comes from m)
        .withColumn(
            "merchant_idx",
            (F.rand(seed=42) * F.lit(N_MERCHANTS)).cast("int")  # 0 .. N_MERCHANTS-1
        )
        .join(
            merchants_idx.select("merchant_idx", "merchant_id"),
            on="merchant_idx",
            how="left"
        )
        .drop("merchant_idx")
)

# CHARGEBACKS (derived from fraud txns)
chargebacks = (
    transactions.filter(F.col("is_fraud") == True)
    .select(
        F.expr("uuid()").alias("cb_id"),
        F.col("txn_id"),
        F.when(F.rand()<0.6, "FRAUD").otherwise("DISPUTE").alias("reason_code"),
        F.from_unixtime(F.unix_timestamp("txn_dt") + (F.rand() * (14*24*60*60)).cast("int")).cast("timestamp").alias("filed_dt"),
        (F.rand()<0.35).cast("boolean").alias("won_dispute")
    )
)

# LOGIN EVENTS
login_events = (
    a.select(
        F.expr("uuid()").alias("event_id"),
        F.col("a.account_id").alias("customer_id"),
        F.expr("uuid()").alias("device_id"),
        F.from_unixtime(F.unix_timestamp() - (F.rand() * (60*24*60*60)).cast("int")).cast("timestamp").alias("event_dt"),
        F.concat(
            (F.rand()*255).cast("int"), F.lit("."),(F.rand()*255).cast("int"),
            F.lit("."),(F.rand()*255).cast("int"), F.lit("."),(F.rand()*255).cast("int")
        ).alias("ip"),
        F.element_at(F.array(*[F.lit(c[1]) for c in countries]), (F.rand()*len(countries)).cast("int")+1).alias("geo"),
        (F.rand()<0.92).cast("boolean").alias("success"),
        (F.rand()<0.35).cast("boolean").alias("mfa_used")
    )
)

# Reassign login IPs so they always come from ip_rep
#addition after issue in PBI
login_events = (
    login_events
        .drop("ip")
        .withColumn(
            "ip_idx",
            (F.rand(seed=123) * F.lit(N_IPS)).cast("int")  # 0..N_IPS-1
        )
        .join(
            ip_rep_idx.select("ip_idx", "ip"),
            on="ip_idx",
            how="left"
        )
        .drop("ip_idx")
)


In [0]:
#Making Data Dirty
transactions_dirty = dirty(transactions, NULL_PCT_SOFT, TYPO_PCT)
login_dirty        = dirty(login_events, NULL_PCT_SOFT, TYPO_PCT)
merchants_dirty    = dirty(merchants, NULL_PCT_SOFT, TYPO_PCT)
devices_dirty      = dirty(devices, NULL_PCT_SOFT, TYPO_PCT)
ip_dirty           = dirty(ip_rep, NULL_PCT_SOFT, TYPO_PCT)
chargebacks_dirty  = dirty(chargebacks, NULL_PCT_SOFT, TYPO_PCT)
accounts_dirty     = dirty(accounts, NULL_PCT_SOFT, TYPO_PCT)


In [0]:
from datetime import date
ingest_dt = "2025-11-03"

outputs = {
  "merchants"     : merchants_dirty,
  "devices"       : devices_dirty,
  "accounts"      : accounts_dirty,
  "ip_reputation" : ip_dirty,
  "transactions"  : transactions_dirty,
  "login_events"  : login_dirty,
  "chargebacks"   : chargebacks_dirty
}

for name, df in outputs.items():
    path = f"{BLOB_BASE}/{name}/ingest_dt={ingest_dt}"
    (df.write
       .mode("overwrite")
       .option("header", "true")
       .option("compression", "gzip")
       .csv(path))
    print("Wrote", path)


Wrote wasbs://bronze@stfrriskneu.blob.core.windows.net/merchants/ingest_dt=2025-11-03
Wrote wasbs://bronze@stfrriskneu.blob.core.windows.net/devices/ingest_dt=2025-11-03
Wrote wasbs://bronze@stfrriskneu.blob.core.windows.net/accounts/ingest_dt=2025-11-03
Wrote wasbs://bronze@stfrriskneu.blob.core.windows.net/ip_reputation/ingest_dt=2025-11-03
Wrote wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03
Wrote wasbs://bronze@stfrriskneu.blob.core.windows.net/login_events/ingest_dt=2025-11-03
Wrote wasbs://bronze@stfrriskneu.blob.core.windows.net/chargebacks/ingest_dt=2025-11-03


In [0]:
# Should list part-*.csv.gz files for today's partition
display(dbutils.fs.ls(f"{BLOB_BASE}/transactions/ingest_dt={ingest_dt}"))

# Quick preview
preview = spark.read.option("header", True).csv(f"{BLOB_BASE}/transactions/ingest_dt={ingest_dt}")
display(preview.limit(5))


path name size modificationTime wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/_SUCCESS _SUCCESS 0 1763449150000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/_committed_4075594431828344539 _committed_4075594431828344539 755 1763449150000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/_committed_444717273710056175 _committed_444717273710056175 762 1763449040000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/_committed_6529668114461184787 _committed_6529668114461184787 388 1762148591000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/_committed_vacuum389551883595925344 _committed_vacuum389551883595925344 95 1763449040000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/_started_4075594431828344539 _started_4075594431828344539 0 1763449148000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/_started_444717273710056175 _started_444717273710056175 0 1763449037000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/part-00000-tid-4075594431828344539-446fe651-58ed-4376-9df8-a31f77282b27-298-1-c000.csv.gz part-00000-tid-4075594431828344539-446fe651-58ed-4376-9df8-a31f77282b27-298-1-c000.csv.gz 9178849 1763449150000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/part-00001-tid-4075594431828344539-446fe651-58ed-4376-9df8-a31f77282b27-299-1-c000.csv.gz part-00001-tid-4075594431828344539-446fe651-58ed-4376-9df8-a31f77282b27-299-1-c000.csv.gz 9236344 1763449150000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/part-00002-tid-4075594431828344539-446fe651-58ed-4376-9df8-a31f77282b27-300-1-c000.csv.gz part-00002-tid-4075594431828344539-446fe651-58ed-4376-9df8-a31f77282b27-300-1-c000.csv.gz 9217464 1763449150000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/part-00003-tid-4075594431828344539-446fe651-58ed-4376-9df8-a31f77282b27-301-1-c000.csv.gz part-00003-tid-4075594431828344539-446fe651-58ed-4376-9df8-a31f77282b27-301-1-c000.csv.gz 9095240 1763449150000

txn_id account_id card_id device_id txn_dt amount currency country_iso city channel entry_mode ip distance_km_prev txn_in_last_min5 is_international is_new_device is_night is_high_amount is_fraud merchant_id 4b1d5e5c-8573-4dbb-9ab9-de61dd77ff85 8525da25-c195-4adf-b7ea-0dad16411ace CARD_8589934592 0f80ed46-158c-4102-b9f6-5c21b8df8d51 2025-11-14T20:58:04.000Z 462.4 EUR IT Mumbai MOBILE null 54.234.251.235 null 0 false false false true false 8d5c3508-94d1-494a-aeb7-00ffb207d89c d9e2247c-8a63-497c-8b03-cd61b47f7a8c a5142b2e-471d-4aee-857d-b664005f134a CARD_8589934593 57fb106e-ccb5-4db6-a118-3e8af888511d 2025-08-27T19:21:23.000Z 37.69 EUR IT Mumbai WEB KEYED 60.31.63.215 813.92 1 false false false false false a6c71a17-3a27-4745-82f7-f5d1845ccb5b 71a2db8a-29b2-4ea7-8494-843cc1047a61 5255ae74-0f70-430e-b4f9-dee792d0a1d0 CARD_8589934594 c923f4e6-52b1-46c3-9fa4-4c8138c562a7 2025-09-06T23:00:23.000Z 371.64 EUR NL Mumbai MOBILE TAP 98.141.250.186 5.45 0 false false false true false b1ccbf57-5a60-4fa0-9012-498be386a758 7517545a-3f5d-4f2b-8e78-0d7472068f0a 20e78028-8fa3-43fa-bb9d-5caba27dae82 CARD_8589934595 3458ed3b-5eef-474d-b669-3728ccf1bc70 2025-08-28T01:07:16.000Z 100.88 EUR GB Madrid WEB CHIP 186.218.89.51 1150.49 3 false true false false false 48afef79-99e6-4d03-ab88-6ea761e71915 be12a070-ac7b-4810-8887-d2c8498f4948 6162ffaa-0260-4a78-a979-9b37d3dec294 CARD_8589934596 70a1d3f5-96dc-4f90-afe8-5afd5e9eeead 2025-09-10T09:53:57.000Z 88.44 EUR IN Berlin WEB SWIPE 22.122.147.32 851.92 3 true false false false false e4ed0e5b-23d8-4822-9ee6-734fa25af30a

In [0]:
from datetime import date
ingest_dt = "2025-11-03"

display(dbutils.fs.ls(f"{BLOB_BASE}/transactions/ingest_dt={ingest_dt}"))
spark.read.option("header", True).csv(f"{BLOB_BASE}/transactions/ingest_dt={ingest_dt}").show(5, truncate=False)


path name size modificationTime wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/_SUCCESS _SUCCESS 0 1763449150000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/_committed_4075594431828344539 _committed_4075594431828344539 755 1763449150000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/_committed_444717273710056175 _committed_444717273710056175 762 1763449040000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/_committed_6529668114461184787 _committed_6529668114461184787 388 1762148591000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/_committed_vacuum389551883595925344 _committed_vacuum389551883595925344 95 1763449040000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/_started_4075594431828344539 _started_4075594431828344539 0 1763449148000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/_started_444717273710056175 _started_444717273710056175 0 1763449037000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/part-00000-tid-4075594431828344539-446fe651-58ed-4376-9df8-a31f77282b27-298-1-c000.csv.gz part-00000-tid-4075594431828344539-446fe651-58ed-4376-9df8-a31f77282b27-298-1-c000.csv.gz 9178849 1763449150000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/part-00001-tid-4075594431828344539-446fe651-58ed-4376-9df8-a31f77282b27-299-1-c000.csv.gz part-00001-tid-4075594431828344539-446fe651-58ed-4376-9df8-a31f77282b27-299-1-c000.csv.gz 9236344 1763449150000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/part-00002-tid-4075594431828344539-446fe651-58ed-4376-9df8-a31f77282b27-300-1-c000.csv.gz part-00002-tid-4075594431828344539-446fe651-58ed-4376-9df8-a31f77282b27-300-1-c000.csv.gz 9217464 1763449150000 wasbs://bronze@stfrriskneu.blob.core.windows.net/transactions/ingest_dt=2025-11-03/part-00003-tid-4075594431828344539-446fe651-58ed-4376-9df8-a31f77282b27-301-1-c000.csv.gz part-00003-tid-4075594431828344539-446fe651-58ed-4376-9df8-a31f77282b27-301-1-c000.csv.gz 9095240 1763449150000

+------------------------------------+------------------------------------+---------------+------------------------------------+------------------------+------+--------+-----------+------+-------+----------+--------------+----------------+----------------+----------------+-------------+--------+--------------+--------+------------------------------------+
|txn_id                              |account_id                          |card_id        |device_id                           |txn_dt                  |amount|currency|country_iso|city  |channel|entry_mode|ip            |distance_km_prev|txn_in_last_min5|is_international|is_new_device|is_night|is_high_amount|is_fraud|merchant_id                         |
+------------------------------------+------------------------------------+---------------+------------------------------------+------------------------+------+--------+-----------+------+-------+----------+--------------+----------------+----------------+----------------+-----------

**Data Ingestion**

Done in ADF. 
